# Rain gauge checks for quality control flags
Covers QC1-7

## Table of contents
[QC1 Percentiles](#QC1---Percentiles)  
[QC2 K-largest](#QC2---K-largest)  
[QC3 Days of week](#QC3---Days-of-week)  
[QC4 Hours of day](#QC4---Hours-of-day)  
[QC5 Intermittency](#QC5---Intermittency)  
[QC6 Breakpoints](#QC6---Breakpoints)  
[QC7 Minimum value change](#QC7---Minimum-value-change)  

See '3.3 Suspect gauges' in Lewis et al. (2021)

In [1]:
import datetime

import numpy as np
import polars as pl
import scipy

In [2]:
def read_metadata(data_path):
    metadata = {}

    with open(data_path, 'r') as f:
        while True:
            key, val = f.readline().strip().split(':', maxsplit=1)
            key = key.lower().replace(' ', '_')
            metadata[key.strip()] = val.strip()
            if key == 'other':
                break
    return metadata

In [3]:
metadata = read_metadata(data_path='../data/gauge_data/DE_02483.txt')
startdate = datetime.datetime.strptime(metadata['start_datetime'], '%Y%m%d%H')
enddate = datetime.datetime.strptime(metadata['end_datetime'], '%Y%m%d%H')

In [4]:
metadata

{'station_id': 'DE_02483',
 'country': 'Germany',
 'original_station_number': '02483',
 'original_station_name': 'NA',
 'path_to_original_data': 'B:/INTENSE data/Original data/Germany/hourly/precipitation/historical/stundenwerte_RR_02483_19951012_20141231_hist.zip,B:/INTENSE data/Original data/Germany/hourly/precipitation/recent/stundenwerte_RR_02483_akt.zip',
 'latitude': '51.1803',
 'longitude': '8.4891',
 'start_datetime': '2006010100',
 'end_datetime': '2010123123',
 'elevation': '839m',
 'number_of_records': '43824',
 'percent_missing_data': '0.00',
 'original_timestep': '1hr',
 'new_timestep': '1hr',
 'original_units': 'mm',
 'new_units': 'mm',
 'time_zone': 'CET',
 'daylight_saving_info': 'NA',
 'no_data_value': '-999',
 'resolution': '0.10',
 'other': ''}

In [5]:
rain_col = f'rain_{metadata['original_units']}'

In [6]:
def get_delta(d1, d2):
    delta = d2 - d1
    return delta


hourly_date_interval = []
delta = get_delta(startdate, enddate+datetime.timedelta(days=1))
for i in range(delta.days * 24):
    hourly_date_interval.append(startdate + datetime.timedelta(hours=i))

In [7]:
gauge_data = pl.read_csv('../data/gauge_data/DE_02483.txt', skip_rows=20)

assert len(gauge_data) == len(hourly_date_interval)

## set time columns
gauge_data = gauge_data.with_columns(time=pl.Series(hourly_date_interval))

## Rename
gauge_data = gauge_data.rename({'Other: ': rain_col})

## Reorder (to look nice)
gauge_data = gauge_data.select(['time', rain_col])

In [8]:
## make no data vals nans
gauge_data = gauge_data.with_columns(pl.when(pl.col(rain_col) == int(metadata['no_data_value'])).then(np.nan).otherwise(pl.col(rain_col)).alias(rain_col))

In [ ]:
gauge_data.head()

time,rain_mm
datetime[μs],f64
2006-01-01 00:00:00,0.9
2006-01-01 01:00:00,0.3
2006-01-01 02:00:00,0.3
2006-01-01 03:00:00,0.0
2006-01-01 04:00:00,0.0


In [10]:
gauge_data.group_by_dynamic('time', every='1d').agg(pl.col(rain_col).mean())[rain_col].plot.line()

alt.Chart(...)

# QC1 - Percentiles 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

#### Notes on testing
Tests need to be checking these lists

In [11]:
perc95 = gauge_data.group_by_dynamic('time', every='1y').agg(pl.quantile(rain_col, .95))
perc99 = gauge_data.group_by_dynamic('time', every='1y').agg(pl.quantile(rain_col, .99))

In [12]:
perc95.filter(pl.col(rain_col) == 0)['time'].dt.year().to_list()

[]

In [13]:
perc99.filter(pl.col(rain_col) == 0)['time'].dt.year().to_list()

[]

In [14]:
perc50 = gauge_data.group_by_dynamic('time', every='1y').agg(pl.quantile(rain_col, .50))
perc50.filter(pl.col(rain_col) == 0)['time'].dt.year().to_list()

[2006, 2007, 2008, 2009, 2010]

# QC2 - K-largest
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

#### Notes:
- can share things with QC1 percentiles

In [15]:
## TODO: Write function
gauge_rain_k1 = gauge_data.group_by_dynamic('time', every='1y').agg(pl.col(rain_col).top_k(1).min())
gauge_rain_k5 = gauge_data.group_by_dynamic('time', every='1y').agg(pl.col(rain_col).top_k(5).min())
gauge_rain_k10 = gauge_data.group_by_dynamic('time', every='1y').agg(pl.col(rain_col).top_k(10).min())

In [16]:
gauge_rain_k10.filter(pl.col(rain_col) == 0)['time'].dt.year().to_list()

[]

In [17]:
gauge_rain_k1600 = gauge_data.group_by_dynamic('time', every='1y').agg(pl.col(rain_col).top_k(1600).min())

In [18]:
gauge_rain_k1600.filter(pl.col(rain_col) == 0)['time'].dt.year().to_list()

[2008]

# QC3 - Days of week
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

In [19]:
gauge_weekday_mean = gauge_data.group_by(pl.col('time').dt.weekday()).agg(pl.col(rain_col).drop_nans().mean())[rain_col]
gauge_weekday_mean

rain_mm
f64
0.248514
0.187893
0.237831
1.991163
1.049387
0.213417
0.262668


In [20]:
gauage_data_mean = gauge_data[rain_col].drop_nans().mean()
gauage_data_mean

0.5987189676936745

In [21]:
t_stat, p_val = scipy.stats.ttest_1samp(gauge_weekday_mean, gauage_data_mean)

In [22]:
t_stat, p_val

(np.float64(-8.807848472080411e-05), np.float64(0.9999325789548257))

# QC4 - Hours of day
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

#### Notes:
- can share functionality with QC3

In [23]:
gauge_hourofday_mean = gauge_data.group_by(pl.col('time').dt.hour()).agg(pl.col(rain_col).drop_nans().mean())[rain_col]
gauge_hourofday_mean

rain_mm
f64
0.482763
0.534137
0.451768
0.480605
1.000645
…
0.560236
0.48963
0.499944


In [24]:
gauage_data_mean = gauge_data[rain_col].drop_nans().mean()
gauage_data_mean

0.5987189676936745

In [25]:
t_stat, p_val = scipy.stats.ttest_1samp(gauge_hourofday_mean, gauage_data_mean)

In [26]:
t_stat, p_val

(np.float64(-0.0005176401871918742), np.float64(0.9995914465158195))

# QC5 - Intermittency 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- using polars and cum_sum checks instead of for loops

#### Potential problems:
- may rely on metadata to determine missing values
- check out 'intermittency period identificiation error' part of code in original implementation

In [27]:
test_data = pl.DataFrame({
    rain_col: [np.nan, np.nan, 1, None, None, 4, None, None, None, 8, None, 9, None, None, 12, None, None]
})

In [28]:
missing_values_mask = (gauge_data[rain_col].is_nan())

In [29]:
print(f'{(missing_values_mask).sum()} missing values')

1046 missing values


In [30]:
# Identify missing values
gauge_data_missing = gauge_data.with_columns(
    (missing_values_mask).alias("is_missing")
)


In [31]:
# Identify group numbers for consecutive nulls
gauge_data_missing_groups = gauge_data_missing.with_columns(
    (pl.when(gauge_data_missing["is_missing"])
     .then((~gauge_data_missing["is_missing"]).cum_sum())
     .otherwise(None)).alias("group")
)

In [32]:
gauge_data_missing_group_counts = gauge_data_missing_groups.group_by("group").agg(
    pl.col("is_missing").sum().alias("count")
)

In [33]:
# Get groups with 2 or more missing values
no_data_period_groups = gauge_data_missing_group_counts.filter(pl.col("count") >= 2)["group"]


In [34]:
## should be 15 for this data
no_data_period_groups

group
u32
670
35050
659
532
3607
…
25585
36586
4794


In [35]:
# Select rows belonging to valid groups
gauge_data_no_data_periods = gauge_data_missing_groups.filter(pl.col("group").is_in(no_data_period_groups))


In [36]:
gauge_data_year_counts = gauge_data_no_data_periods.select(pl.col("time").dt.year()).to_series().value_counts()

In [37]:
gauge_data_year_counts

time,count
i32,u32
2010,1008
2006,22
2008,2


In [38]:
gauge_data_year_counts.filter(pl.col('count') >= 5)['time'].to_list()

[2010, 2006]

# QC6 - Breakpoints
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`:
- Use python implementation of Pettitt test instead of rpy

##### Notes for testing:
- could compare results to this package: https://github.com/mmhs013/pyHomogeneity/tree/master
- Need to put some NaNs in and check it still works. 
- Also NaN values may mess up 

In [39]:
def pettitt_test(X):
    """
    Pettitt test calculated following Pettitt (1979): https://www.jstor.org/stable/2346729?seq=4#metadata_info_tab_contents

    TAKEN FROM: https://stackoverflow.com/questions/58537876/how-to-run-standard-normal-homogeneity-test-for-a-time-series-data
    """

    T = len(X)
    U = []
    for t in range(T): # t is used to split X into two subseries
        X_stack = np.zeros((t, len(X[t:]) + 1), dtype=int)
        X_stack[:,0] = X[:t] # first column is each element of the first subseries
        X_stack[:,1:] = X[t:] # all rows after the first element are the second subseries
        U.append(np.sign(X_stack[:,0] - X_stack[:,1:].transpose()).sum()) # sign test between each element of the first subseries and all elements of the second subseries, summed.

    tau = np.argmax(np.abs(U)) # location of change (first data point of second sub-series)
    K = np.max(np.abs(U))
    p = 2 * np.exp(-6 * K**2 / (T**3 + T**2))
    return (tau, p)

In [40]:
gauge_data_daily_upsample = gauge_data.upsample('time', every='1d')

In [41]:
%%time
## 5 seconds
tau, p_val = pettitt_test(gauge_data_daily_upsample[rain_col].fill_nan(0.0))

CPU times: user 2.77 s, sys: 1.92 s, total: 4.69 s
Wall time: 4.69 s


In [42]:
tau, p_val

(np.int64(682), np.float64(1.343422289290123))

In [43]:
import pyhomogeneity

In [44]:
%%time
## 5 seconds
result = pyhomogeneity.pettitt_test(gauge_data_daily_upsample[rain_col].fill_nan(0.0))
result

CPU times: user 4.07 s, sys: 59.6 ms, total: 4.13 s
Wall time: 4.13 s


Pettitt_Test(h=np.False_, cp=np.int64(826), p=np.float64(0.89175), U=np.float64(25480.0), avg=mean(mu1=np.float64(1.4901937046004843), mu2=np.float64(0.3135)))

# QC7 - Minimum value change 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

#### Potential problems:
- relies on resolution given in metadata

#### Notes on testing:
- 

In [45]:
gauge_data_nonzero = gauge_data.filter(pl.col(rain_col) > 0)

In [46]:
gauge_min_by_year = gauge_data_nonzero.group_by_dynamic(pl.col('time'), every='1y').agg(pl.col(rain_col).min())
gauge_min_by_year

time,rain_mm
datetime[μs],f64
2006-01-01 00:00:00,0.2
2007-01-01 00:00:00,0.1
2008-01-01 00:00:00,0.1
2009-01-01 00:00:00,0.1
2010-01-01 00:00:00,0.2


In [47]:
resolution = float(metadata['resolution'])

In [48]:
non_res_years = gauge_min_by_year.filter(pl.col(rain_col) != resolution)

In [49]:

non_res_years['time'].dt.year().to_list()

[2006, 2010]